In [1]:
# Install the necessary packages
! pip install presidio-analyzer presidio-anonymizer -q
! python -m spacy download en_core_web_lg -q

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
! guardrails hub install hub://guardrails/detect_pii

Installing hub://guardrails/detect_pii...
[    ] Fetching manifest
[=   ] Fetching manifest
[==  ] Fetching manifest
[=== ] Fetching manifest
[ ===] Fetching manifest
[  ==] Fetching manifest
[    ] Fetching manifest
[   =] Fetching manifest
[  ==] Fetching manifest
[ ===] Fetching manifest
[====] Fetching manifest
[=== ] Fetching manifest
[=   ] Fetching manifest
[    ] Fetching manifest
[=   ] Fetching manifest
[==  ] Fetching manifest
[=== ] Fetching manifest
[ ===] Fetching manifest
[ ===] Fetching manifest

[    ] Downloading dependencies
[=   ] Downloading dependencies
[==  ] Downloading dependencies
[=== ] Downloading dependencies
[ ===] Downloading dependencies
[  ==] Downloading dependencies
[    ] Downloading dependencies
[   =] Downloading dependencies
[  ==] Downloading dependencies
[ ===] Downloading dependencies
[====] Downloading dependencies
[=== ] Downloading dependencies
[=   ] Downloading dependencies
[    ] Downloading dependencies
[=   ] Downloading dependencies
[=

In [3]:
# Import the guardrails package
from guardrails.hub import DetectPII
import guardrails as gd
from rich import print

c:\Users\Leapfrog\Desktop\guardrail\guardrail\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# It can be passed either durring intialization or later through the metadata argument in parse method.

# One can also pass in a list of entities supported by Presidio to the `pii_entities` argument.
guard = gd.Guard().use(DetectPII(pii_entities="pii", on_fail="fix"))

In [5]:
# Parse the text
text = "My email address is sizaa@gojoo.com, and my phone number is 1234567890"
output = guard.parse(
    llm_output=text,
)

# Print the output
print(output)

c:\Users\Leapfrog\Desktop\guardrail\guardrail\Lib\site-packages\guardrails\validator_service\__init__.py:85: UserWarning: Could not obtain an event loop. Falling back to synchronous validation.
  warnings.warn(


ValidationOutcome(
    call_id='1486461136288',
    raw_llm_output='My email address is sizaa@gojoo.com, and my phone number is 1234567890',
    validation_summaries=[
        ValidationSummary(
            validator_name='DetectPII',
            validator_status='fail',
            property_path='$',
            failure_reason='The following text in your response contains PII:\nMy email address is sizaa@gojoo.com,
and my phone number is 1234567890',
            error_spans=[
                ErrorSpan(start=20, end=35, reason='PII detected in sizaa@gojoo.com'),
                ErrorSpan(start=60, end=70, reason='PII detected in 1234567890')
            ]
        )
    ],
    validated_output='My email address is <EMAIL_ADDRESS>, and my phone number is <PHONE_NUMBER>',
    reask=None,
    validation_passed=True,
    error=None
)

In [7]:
# Let's test with passing through metadata for the same guard object
# This will take precendence over the entities passed in during initialization
output = guard.parse(
    llm_output=text,
    metadata={"pii_entities": ["EMAIL_ADDRESS"]},
)

# Print the output
print(output)

ValidationOutcome(
    call_id='1486470611904',
    raw_llm_output='My email address is demo@xyz.com, and my account number is 1234789012367654.',
    validation_summaries=[
        ValidationSummary(
            validator_name='DetectPII',
            validator_status='fail',
            property_path='$',
            failure_reason='The following text in your response contains PII:\nMy email address is demo@xyz.com, 
and my account number is 1234789012367654.',
            error_spans=[ErrorSpan(start=20, end=32, reason='PII detected in demo@xyz.com')]
        )
    ],
    validated_output='My email address is <EMAIL_ADDRESS>, and my account number is 1234789012367654.',
    reask=None,
    validation_passed=True,
    error=None
)

In [8]:
# Create a new guard object
guard = gd.Guard().use(DetectPII(pii_entities="spi", on_fail="fix"))

In [9]:
# Parse text
text = "My email address is demo@xyz.com, and my account number is 1234789012367654."

output = guard.parse(
    llm_output=text,
)

# Print the output
print(output)

ValidationOutcome(
    call_id='1486453644160',
    raw_llm_output='My email address is demo@xyz.com, and my account number is 1234789012367654.',
    validation_summaries=[
        ValidationSummary(
            validator_name='DetectPII',
            validator_status='fail',
            property_path='$',
            failure_reason='The following text in your response contains PII:\nMy email address is demo@xyz.com, 
and my account number is 1234789012367654.',
            error_spans=[ErrorSpan(start=59, end=75, reason='PII detected in 1234789012367654')]
        )
    ],
    validated_output='My email address is demo@xyz.com, and my account number is <US_BANK_NUMBER>.',
    reask=None,
    validation_passed=True,
    error=None
)

In [10]:
# Another example
text = "My PANNUMBER is 923756789 and my driver's license number is 87651239"

output = guard.parse(
    llm_output=text,
    metadata={"pii_entities": ["US_ITIN", "US_DRIVER_LICENSE"]},
)

# Print the output
print(output)

ValidationOutcome(
    call_id='1486453641040',
    raw_llm_output="My PANNUMBER is 923756789 and my driver's license number is 87651239",
    validation_summaries=[
        ValidationSummary(
            validator_name='DetectPII',
            validator_status='fail',
            property_path='$',
            failure_reason="The following text in your response contains PII:\nMy PANNUMBER is 923756789 and my 
driver's license number is 87651239",
            error_spans=[
                ErrorSpan(start=16, end=25, reason='PII detected in 923756789'),
                ErrorSpan(start=60, end=68, reason='PII detected in 87651239')
            ]
        )
    ],
    validated_output="My PANNUMBER is <US_ITIN> and my driver's license number is <US_DRIVER_LICENSE>",
    reask=None,
    validation_passed=True,
    error=None
)